# MongoDB via Mongo Shell

In [2]:
from pymongo import MongoClient
import pprint as pp
from pymongo import ASCENDING, DESCENDING
from bson.objectid import ObjectId
from datetime import datetime

# Outline

- Establish Connection to Server
- Show Databases
- Import Data
- Select Imported DB
- Show Collections
- Sample Documents
- Get Record Counts
- Get List of Distinct Fields
- Get Number of Distinct Values by Field
- Get List of Unique Values for a Field
- Get Count of Unique Values by Value for a Field
- CRUD Operations
	- Delete and Create
	- Read
	- Update
- Indexes
	- View Indexes
	- Create Indexes

# Establish Connection to Server

In [3]:
client = MongoClient("mongodb://localhost:27017/")

# Show Databases

In [4]:
print(client.list_database_names())

['admin', 'clickstream', 'config', 'local']


# Import Data

### Drop <code>clickstream</code> if Exists

In [5]:
db_name = "clickstream"
client.drop_database(db_name)

### Import From File

In [ ]:
HOST = "localhost"
PORT = "27017"
DBNAME = "clickstream"
IMPORT_FILE_FOLDER = r"C:\Users\patwh\Downloads"
BSON_FILE_NAME = "clicks"
JSON_FILE_NAME = "clicks.metadata"
bson_file = f"{IMPORT_FILE_FOLDER}\\{BSON_FILE_NAME}.bson"
json_file = f"{IMPORT_FILE_FOLDER}\\{JSON_FILE_NAME}.json"
collection_bson = BSON_FILE_NAME
collection_json = JSON_FILE_NAME

!mongorestore --host {HOST}:{PORT} --db {DBNAME} --collection {collection_bson} --drop "{bson_file}"
!mongoimport --host {HOST}:{PORT} --db {DBNAME} --collection {collection_json} --drop --type json "{json_file}"

In [ ]:
# 2025-06-03T17:04:39.900-0600	finished restoring clickstream.clicks (6100000 documents, 0 failures)
# 2025-06-03T17:04:39.900-0600	no indexes to restore for collection clickstream.clicks
# 2025-06-03T17:04:39.900-0600	6100000 document(s) restored successfully. 0 document(s) failed to restore.
# 2025-06-03T17:04:40.598-0600	connected to: mongodb://localhost:27017/
# 2025-06-03T17:04:40.601-0600	dropping: clickstream.clicks.metadata
# 2025-06-03T17:04:40.641-0600	1 document(s) imported successfully. 0 document(s) failed to import.

# Select Imported DB

In [16]:
db_name = "clickstream"
db = client[db_name]

# Show Collections

In [17]:
collections = db.list_collection_names()
print(collections)

['clicks', 'clicks.metadata']


# Sample Documents

In [19]:
db.clicks.find_one()

{'_id': ObjectId('60df1029ad74d9467c91a932'),
 'webClientID': 'WI100000244987',
 'VisitDateTime': datetime.datetime(2018, 5, 25, 4, 51, 14, 179000),
 'ProductID': 'Pr100037',
 'Activity': 'click',
 'device': {'Browser': 'Firefox', 'OS': 'Windows'},
 'user': {'City': 'Colombo', 'Country': 'Sri Lanka'}}

In [21]:
db.clicks.metadata.find_one()

{'_id': ObjectId('683f7f88add779e2658af02a'),
 'indexes': [{'v': 2, 'key': {'_id': 1}, 'name': '_id_'}],
 'uuid': 'ee6da5fe5bdf42b2bc3cecee40723af6',
 'collectionName': 'clicks'}

# Get Record Counts

In [23]:
db.clicks.count_documents({})

6100000

In [25]:
db.clicks.metadata.count_documents({})

1

### Count of Unique Values (Field device.Browser)

In [26]:
len(db.clicks.distinct("device.Browser"))

82

# Get List of Distinct Fields

In [27]:
fields = set()
for doc in db.clicks.find().limit(1000000):
    fields.update(doc.keys())
print(list(fields))

['user', 'webClientID', '_id', 'ProductID', 'Activity', 'device', 'VisitDateTime']


Including nested fields:

In [28]:
def extract_fields(obj, prefix, fields):
    for key, value in obj.items():
        full_key = f"{prefix}.{key}" if prefix else key
        fields.add(full_key)
        
        # recurse into nested dictionaries
        if isinstance(value, dict):
            extract_fields(value, full_key, fields)

# initialize set of field paths
fields = set()

# iterate over documents
for doc in db.clicks.find().limit(1000000):
    extract_fields(doc, "", fields)

# print unique field paths
print(list(fields))

['user', 'user.City', 'webClientID', '_id', 'device.Browser', 'device.OS', 'ProductID', 'user.Country', 'Activity', 'device', 'VisitDateTime', 'user.UserID']


# Get Number of Distinct Values by Field

#### Hard Coded

In [31]:
collection = db['clicks']

fields = [
    '_id',
    'webClientID',
    'VisitDateTime',
    'ProductID',
    'Activity',
    'device',
    'device.Browser',
    'device.OS',
    'user',
    'user.City',
    'user.Country',
    'user.UserID'
]

# array to store results
results = []

# loop through fields
for field in fields:
    pipeline = [

        # group by field
        {"$group": {"_id": f"${field}"}},

        # get count of group
        {"$group": {"_id": None, "count": {"$sum": 1}}}
    ]

    # run collection through pipeline, convert the result to array
    result = list(collection.aggregate(pipeline))

    # if doc exists, extract count
    count = result[0]["count"] if result else 0

    # append results
    results.append({"field": field, "count": count})

# sort results descending by count
results.sort(key=lambda x: x["count"], reverse=True)

# print sorted results
for item in results:
    print(f"{item['field']}: {item['count']} unique values")

_id: 6100000 unique values
VisitDateTime: 6089023 unique values
webClientID: 1091455 unique values
user: 72162 unique values
user.UserID: 34051 unique values
user.City: 26260 unique values
ProductID: 10938 unique values
user.Country: 222 unique values
device: 151 unique values
device.Browser: 82 unique values
device.OS: 18 unique values
Activity: 2 unique values


#### Dynamic

In [ ]:
# print("=== Starting Unique Field Count ===")

# # Recursively extract field paths
# def extract_fields(obj, prefix="", fields=None):
#     if fields is None:
#         fields = set()
#     for key, value in obj.items():
#         full_key = f"{prefix}.{key}" if prefix else key
#         fields.add(full_key)
#         if isinstance(value, dict):
#             extract_fields(value, full_key, fields)
#     return fields

# # Get all field paths from documents
# field_set = set()
# for doc in db.clicks.find():
#     field_set.update(extract_fields(doc))

# # Count distinct values for each field
# results = []
# for field in field_set:
#     try:
#         pipeline = [
#             {"$group": {"_id": f"${field}"}},
#             {"$group": {"_id": None, "count": {"$sum": 1}}}
#         ]
#         res = list(db.clicks.aggregate(pipeline))
#         count = res[0]["count"] if res else 0
#         results.append({"field": field, "count": count})
#     except Exception as e:
#         print(f"Error counting values for {field}: {str(e)}")
#         results.append({"field": field, "count": 0})

# # Sort results by count in descending order
# results.sort(key=lambda x: x["count"], reverse=True)

# # Print results with numbering
# for i, r in enumerate(results, 1):
#     print(f"{i}. {r['field']}: {r['count']} unique values")

# Get List of Unique Values for a Field

In [32]:
db.clicks.distinct("device.Browser")

['AdsBot-Google',
 'AhrefsBot',
 'Amazon Silk',
 'Android',
 'AppEngine-Google',
 'Apple Mail',
 'BingPreview',
 'BlackBerry WebKit',
 'Chrome',
 'Chrome Mobile',
 'Chrome Mobile WebView',
 'Chrome Mobile iOS',
 'Chromium',
 'Coc Coc',
 'Coveobot',
 'Crosswalk',
 'Dragon',
 'DuckDuckBot',
 'Edge',
 'Edge Mobile',
 'Electron',
 'Epiphany',
 'Facebook',
 'FacebookBot',
 'Firefox',
 'Firefox Mobile',
 'Firefox iOS',
 'HbbTV',
 'HeadlessChrome',
 'HubSpot Crawler',
 'IE',
 'IE Mobile',
 'Iceweasel',
 'Iron',
 'JobBot',
 'Jooblebot',
 'K-Meleon',
 'Kindle',
 'Konqueror',
 'Magus Bot',
 'Mail.ru Chromium Browser',
 'Maxthon',
 'Mobile Safari',
 'Mobile Safari UI/WKWebView',
 'MobileIron',
 'NetFront',
 'Netscape',
 'Opera',
 'Opera Coast',
 'Opera Mini',
 'Opera Mobile',
 'Other',
 'PagePeeker',
 'Pale Moon',
 'PetalBot',
 'PhantomJS',
 'Pinterest',
 'Puffin',
 'Python Requests',
 'QQ Browser',
 'QQ Browser Mobile',
 'Radius Compliance Bot',
 'Safari',
 'Samsung Internet',
 'SeaMonkey',
 'Se

# Get Count of Unique Values by Value for a Field

In [38]:
result = db.clicks.aggregate([
    {"$group": {"_id": "$device.Browser", "count": {"$sum": 1}}}
])
counts = {doc["_id"]: doc["count"] for doc in result}
sorted_counts = dict(sorted(counts.items(), key=lambda x: x[1], reverse=True))

pp.pprint(sorted_counts)

{'AdsBot-Google': 312,
 'AhrefsBot': 229,
 'Amazon Silk': 376,
 'Android': 19,
 'AppEngine-Google': 22,
 'Apple Mail': 3038,
 'BingPreview': 33,
 'BlackBerry WebKit': 2,
 'Chrome': 4360498,
 'Chrome Mobile': 788991,
 'Chrome Mobile WebView': 25381,
 'Chrome Mobile iOS': 28031,
 'Chromium': 629,
 'Coc Coc': 4506,
 'Coveobot': 8,
 'Crosswalk': 27,
 'Dragon': 105,
 'DuckDuckBot': 1,
 'Edge': 5180,
 'Edge Mobile': 17,
 'Electron': 13,
 'Epiphany': 15,
 'Facebook': 5874,
 'FacebookBot': 12,
 'Firefox': 388766,
 'Firefox Mobile': 7040,
 'Firefox iOS': 884,
 'HbbTV': 4,
 'HeadlessChrome': 76595,
 'HubSpot Crawler': 1,
 'IE': 3381,
 'IE Mobile': 22,
 'Iceweasel': 2,
 'Iron': 156,
 'JobBot': 208,
 'Jooblebot': 466,
 'K-Meleon': 2,
 'Kindle': 3,
 'Konqueror': 1,
 'Magus Bot': 1,
 'Mail.ru Chromium Browser': 5,
 'Maxthon': 139,
 'Mobile Safari': 93241,
 'Mobile Safari UI/WKWebView': 3038,
 'MobileIron': 3,
 'NetFront': 1,
 'Netscape': 2,
 'Opera': 62522,
 'Opera Coast': 2,
 'Opera Mini': 12,
 'Op

# CRUD Operations

## Delete and Create

### Delete the Last Record

Remove the last record from <code>clicks</code> and re-insert it.

First capture in variable:

In [39]:
last_doc = db.clicks.find().sort("_id", -1).limit(1).next()
pp.pprint(last_doc)

{'Activity': 'click',
 'ProductID': 'Pr101251',
 'VisitDateTime': datetime.datetime(2018, 5, 26, 11, 51, 44, 263000),
 '_id': ObjectId('60df129dad74d9467ceebd51'),
 'device': {'Browser': 'Chrome', 'OS': 'Windows'},
 'user': {'City': 'Vijayawada', 'Country': 'India'},
 'webClientID': 'WI100000118333'}


Delete it:

In [40]:
db.clicks.delete_one({"_id": last_doc["_id"]})

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

### Insert it Back In

In [41]:
db.clicks.insert_one(last_doc)

InsertOneResult(ObjectId('60df129dad74d9467ceebd51'), acknowledged=True)

### Delete the Last 5 Records

In [42]:
last_docs = list(db.clicks.find().sort("_id", -1).limit(5))
ids_to_delete = [doc["_id"] for doc in last_docs]

In [43]:
db.clicks.delete_many({"_id": {"$in": ids_to_delete}})

DeleteResult({'n': 5, 'ok': 1.0}, acknowledged=True)

### Insert Them Back In

In [44]:
db.clicks.insert_many(last_docs)

InsertManyResult([ObjectId('60df129dad74d9467ceebd51'), ObjectId('60df129dad74d9467ceebd50'), ObjectId('60df129dad74d9467ceebd4f'), ObjectId('60df129dad74d9467ceebd4e'), ObjectId('60df129dad74d9467ceebd4d')], acknowledged=True)

## Read

### Filter to <code>_id</code> Equal to <code>60df129dad74d9467ceebd51</code>

In [49]:
db.clicks.find_one({"_id": ObjectId("60df129dad74d9467ceebd51")})

{'_id': ObjectId('60df129dad74d9467ceebd51'),
 'webClientID': 'WI100000118333',
 'VisitDateTime': datetime.datetime(2018, 5, 26, 11, 51, 44, 263000),
 'ProductID': 'Pr101251',
 'Activity': 'click',
 'device': {'Browser': 'Chrome', 'OS': 'Windows'},
 'user': {'City': 'Vijayawada', 'Country': 'India'}}

### Find First Record Where <code>device.Browser</code> is not Firefox

In [50]:
db.clicks.find_one({"device.Browser": "Firefox"})

{'_id': ObjectId('60df1029ad74d9467c91a932'),
 'webClientID': 'WI100000244987',
 'VisitDateTime': datetime.datetime(2018, 5, 25, 4, 51, 14, 179000),
 'ProductID': 'Pr100037',
 'Activity': 'click',
 'device': {'Browser': 'Firefox', 'OS': 'Windows'},
 'user': {'City': 'Colombo', 'Country': 'Sri Lanka'}}

### Find First 2 Records Where <code>device.Browser</code> is not Firefox

In [54]:
records = list(db.clicks.find({"device.Browser": {"$ne": "Firefox"}}).limit(2))
pp.pprint(records)

[{'Activity': 'pageload',
  'ProductID': 'Pr100872',
  'VisitDateTime': datetime.datetime(2018, 5, 25, 5, 6, 3, 700000),
  '_id': ObjectId('60df1029ad74d9467c91a933'),
  'device': {'Browser': 'Chrome Mobile', 'OS': 'Android'},
  'user': {},
  'webClientID': 'WI10000061461'},
 {'Activity': 'click',
  'ProductID': 'Pr100457',
  'VisitDateTime': datetime.datetime(2018, 5, 17, 11, 51, 9, 265000),
  '_id': ObjectId('60df1029ad74d9467c91a934'),
  'device': {'Browser': 'Chrome', 'OS': 'Linux'},
  'user': {'City': 'Ottawa', 'Country': 'Canada'},
  'webClientID': 'WI10000075748'}]


### Find First 2 Records Where <code>VisitDateTime</code> > 5/20/2018

In [58]:
records = list(db.clicks.find({
    "VisitDateTime": {"$gt": datetime(2018, 5, 20)}
}).limit(2))

pp.pprint(records)

[{'Activity': 'click',
  'ProductID': 'Pr100037',
  'VisitDateTime': datetime.datetime(2018, 5, 25, 4, 51, 14, 179000),
  '_id': ObjectId('60df1029ad74d9467c91a932'),
  'device': {'Browser': 'Firefox', 'OS': 'Windows'},
  'user': {'City': 'Colombo', 'Country': 'Sri Lanka'},
  'webClientID': 'WI100000244987'},
 {'Activity': 'pageload',
  'ProductID': 'Pr100872',
  'VisitDateTime': datetime.datetime(2018, 5, 25, 5, 6, 3, 700000),
  '_id': ObjectId('60df1029ad74d9467c91a933'),
  'device': {'Browser': 'Chrome Mobile', 'OS': 'Android'},
  'user': {},
  'webClientID': 'WI10000061461'}]


### Get the Minimum and Maximum <code>VisitDateTime</code>

In [59]:
result = list(db.clicks.aggregate([
    {"$group": {
        "_id": None,
        "minVisitDateTime": {"$min": "$VisitDateTime"},
        "maxVisitDateTime": {"$max": "$VisitDateTime"}
    }}
]))

pp.pprint(result)

[{'_id': None,
  'maxVisitDateTime': datetime.datetime(2018, 5, 27, 23, 59, 59, 576000),
  'minVisitDateTime': datetime.datetime(2018, 5, 7, 0, 0, 1, 190000)}]


### Get Count of Records Where <code>VisitDateTime</code> is Greater Than 5/20/2018

In [60]:
count = db.clicks.count_documents({
    "VisitDateTime": {"$gt": datetime(2018, 5, 20)}
})
print(count)

2453050


### Get Count of Records Where <code>user.Country</code> is <code>India</code> or <code>United States</code>

#### Using <code>$or</code>

In [61]:
count = db.clicks.count_documents({
    "$or": [
        {"user.Country": "India"},
        {"user.Country": "United States"}
    ]
})
print(count)

3497232


#### Using <code>$in</code>

In [62]:
count = db.clicks.count_documents({
    "user.Country": {"$in": ["India", "United States"]}
})
print(count)

3497232


### Get Count of Records Where <code>user.Country</code> is Neither <code>India</code> Nor <code>United States</code>

#### Using <code>$and</code>

In [63]:
count = db.clicks.count_documents({
    "$and": [
        {"user.Country": {"$ne": "India"}},
        {"user.Country": {"$ne": "United States"}}
    ]
})
print(count)

2602768


#### Using `$not` and `$in`

In [64]:
count = db.clicks.count_documents({
    "user.Country": {"$not": {"$in": ["India", "United States"]}}
})
print(count)

2602768


#### Using <code>$nin</code>

In [65]:
count = db.clicks.count_documents({
    "user.Country": {"$nin": ["India", "United States"]}
})
print(count)

2602768


### Get Count of Records with <code>user.UserID</code>

In [66]:
count = db.clicks.count_documents({
  "user.UserID": { "$exists": "true", "$ne": "null" }
})
print(count)

602293


## Update

### Update <code>device.Browser</code> for Record <code>60df129dad74d9467ceebd51</code> to <code>Firefox</code>

In [18]:
result = db.clicks.update_one(
    {"_id": ObjectId("60df129dad74d9467ceebd51")},
    {"$set": {"device.Browser": "Firefox"}}
)
print(result.modified_count)

1


Set it back to original state for accuracy:

In [19]:
result = db.clicks.update_one(
    {"_id": ObjectId("60df129dad74d9467ceebd51")},
    {"$set": {"device.Browser": "Chrome"}}
)
print(result.modified_count)

1


### Update <code>device.Browser</code> Records to be <code>Firefox</code> if Set to <code>Firefox iOS</code>

In [ ]:
# result = db.clicks.update_many(
#     {"device.Browser": "Firefox iOS"},
#     {"$set": {"device.Browser": "Firefox"}}
# )
# print(result.modified_count)

# Create New Field

### Add Field Called <code>NewField</code> to First 1000 Records, Set Value to <code>Default</code>

In [20]:
for doc in db.clicks.find().limit(1000):
    db.clicks.update_one(
        {"_id": doc["_id"]},
        {"$set": {"NewField": "Default"}}
    )

In [21]:
db.clicks.find_one({"NewField": "Default"})

{'_id': ObjectId('60df1029ad74d9467c91a932'),
 'webClientID': 'WI100000244987',
 'VisitDateTime': datetime.datetime(2018, 5, 25, 4, 51, 14, 179000),
 'ProductID': 'Pr100037',
 'Activity': 'click',
 'device': {'Browser': 'Firefox', 'OS': 'Windows'},
 'user': {'City': 'Colombo', 'Country': 'Sri Lanka'},
 'NewField': 'Default'}

### Remove the Added Field

In [22]:
result = db.clicks.update_many(
    {"NewField": {"$exists": True}},
    {"$unset": {"NewField": ""}}
)
print(result.modified_count)

1000


# Indexes

## View Indexes

In [23]:
indexes = db.clicks.index_information()
print(indexes)

{'_id_': {'v': 2, 'key': [('_id', 1)]}}


In [28]:
result = db.clicks.metadata.find_one()
pp.pprint(result)

{'_id': ObjectId('683f930cd9991db2f0cb8a75'),
 'collectionName': 'clicks',
 'indexes': [{'key': {'_id': 1}, 'name': '_id_', 'v': 2}],
 'uuid': 'ee6da5fe5bdf42b2bc3cecee40723af6'}


## Create Indexes

In [29]:
db.clicks.create_index({ "device.OS": 1 })

'device.OS_1'

In [30]:
db.clicks.metadata.create_index({ "device.OS": 1 })

'device.OS_1'